In [2]:
# perform imports
import pandas as pd
import datetime as dt

pd.options.mode.chained_assignment = None

In [3]:
# load dataset with the appropriate encoding; other encodings do not load properly
data_raw = pd.read_csv('us_accidents_weather_data.csv', encoding="ISO-8859-1")

In [4]:
# drop rows with a null weather timestamp
data_cleaned = data_raw.dropna(subset='Weather_Timestamp')
# set remaining weather timestamps to datetime
data_cleaned['Weather_Timestamp'] = data_cleaned['Weather_Timestamp'].astype('datetime64[ns]')

In [5]:
# drop rows without valuable information
data_cleaned = data_cleaned.dropna(subset=['Temperature(F)', 'Wind_Chill(F)',
                                           'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
                                           'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition'],
                                   how='all')

In [6]:
# estimate the Sunrise_Sunset value for null columns based on typical times of day/night
mask1 = data_cleaned['Weather_Timestamp'].dt.time >= dt.time(7, 30)
mask2 = data_cleaned['Weather_Timestamp'].dt.time <= dt.time(19, 30)
mask3 = data_cleaned['Sunrise_Sunset'].isna()

data_cleaned.loc[mask1 & mask2 & mask3, 'Sunrise_Sunset'] = 'Day'
data_cleaned['Sunrise_Sunset'] = data_cleaned['Sunrise_Sunset'].fillna('Night')

In [7]:
# temperature follows a distribution that seems reasonable to sample from randomly
temp_sample = data_cleaned['Temperature(F)'].dropna().sample(n=len(data_cleaned), replace=True)
temp_sample.index = data_cleaned.index
data_cleaned['Temperature(F)'] = data_cleaned['Temperature(F)'].fillna(temp_sample)

In [8]:
# wind chill follows a distribution that seems reasonable to sample from randomly
wind_chill_sample = data_cleaned['Wind_Chill(F)'].dropna().sample(n=len(data_cleaned), replace=True)
wind_chill_sample.index = data_cleaned.index
data_cleaned['Wind_Chill(F)'] = data_cleaned['Wind_Chill(F)'].fillna(wind_chill_sample)

In [9]:
# humidity follows a distribution that seems reasonable to sample from randomly
humidity_sample = data_cleaned['Humidity(%)'].dropna().sample(n=len(data_cleaned), replace=True)
humidity_sample.index = data_cleaned.index
data_cleaned['Humidity(%)'] = data_cleaned['Humidity(%)'].fillna(humidity_sample)

In [10]:
# the distribution for pressure is irregular, so filling NAs with the median is safest
pressure_median = data_cleaned['Pressure(in)'].median()
data_cleaned['Pressure(in)'] = data_cleaned['Pressure(in)'].fillna(pressure_median)

In [11]:
# the distribution for visibility is irregular, so filling NAs with the median is safest
visibility_median = data_cleaned['Visibility(mi)'].median()
data_cleaned['Visibility(mi)'] = data_cleaned['Visibility(mi)'].fillna(visibility_median)

In [12]:
# the distribution for wind speed is irregular, so filling NAs with the median is safest
wind_speed_median = data_cleaned['Wind_Speed(mph)'].median()
data_cleaned['Wind_Speed(mph)'] = data_cleaned['Wind_Speed(mph)'].fillna(wind_speed_median)

In [13]:
# the distribution for precipitation is irregular, so filling NAs with the median is safest
precip_median = data_cleaned['Precipitation(in)'].median()
data_cleaned['Precipitation(in)'] = data_cleaned['Precipitation(in)'].fillna(precip_median)

In [14]:
# weather condition follows a distribution that seems reasonable to sample from randomly
weather_sample = data_cleaned['Weather_Condition'].dropna().sample(n=len(data_cleaned), replace=True)
weather_sample.index = data_cleaned.index
data_cleaned['Weather_Condition'] = data_cleaned['Weather_Condition'].fillna(weather_sample)

In [15]:
# confirm that all NAs have been removed
print(f"Number of remaining NAs: {data_cleaned.isna().any(axis=1).sum()}")

Number of remaining NAs: 0


In [19]:
# save the resulting dataframe as a CSV
data_cleaned.to_csv('us_accidents_weather_data_CLEAN.csv')